# Personal Agendas Complete Pipeline with 4 Steps
This notebook submits the complete Personal Agendas pipeline including:
- Step 1: Data Preparation (Registration, Scan, Session)
- Step 2: Neo4j Preparation (Visitors, Sessions, Relationships)
- Step 3: Session Embedding
- Step 4: Recommendations


In [ ]:
# Import required libraries
import os
import sys
import json
from pathlib import Path
from datetime import datetime

import yaml

# Azure ML imports
from azure.ai.ml import MLClient, command, Input, Output
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Environment, AmlCompute
from azure.ai.ml.dsl import pipeline
from azure.identity import DefaultAzureCredential
from azure.identity import ClientSecretCredential
from azure.core.exceptions import ResourceNotFoundError

from dotenv import load_dotenv

DEFAULT_PA_ENV_VERSION = 11

# Load environment variables
load_dotenv()
os.environ.setdefault("PA_ENV_VERSION", str(DEFAULT_PA_ENV_VERSION))

print("="*60)
print("AZURE ML PIPELINE COMPLETE")
print("="*60)

# Configuration
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")
tenant_id = os.getenv("AZURE_TENANT_ID")
subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
workspace_name = os.getenv("AZUREML_WORKSPACE_NAME")



print(f"Azure ML SDK Version: {sys.version}")
print(f"Current time: {datetime.now().isoformat()}")
print(f"Target Azure ML env version: {os.getenv('PA_ENV_VERSION')}")

AZURE ML PIPELINE COMPLETE
Azure ML SDK Version: 3.10.18 (main, Jun  5 2025, 13:14:17) [GCC 11.2.0]
Current time: 2025-12-19T18:46:51.190994
Target Azure ML env version: 10


In [2]:
load_dotenv()

True

In [3]:
# IMPORTANT: Configure Neo4j credentials
# Option 1: Load from .env file


# Option 2: Set directly here (replace with your actual credentials)
# Uncomment and update these lines:
# os.environ["NEO4J_URI"] = "neo4j+s://your-instance.databases.neo4j.io"
# os.environ["NEO4J_USERNAME"] = "neo4j"
# os.environ["NEO4J_PASSWORD"] = "your-password"

neo4j_uri = os.getenv("NEO4J_URI")
neo4j_username = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")

if not all([neo4j_uri, neo4j_username, neo4j_password]):
    print("WARNING: Neo4j credentials not found")
    print("Please set NEO4J_URI, NEO4J_USERNAME, and NEO4J_PASSWORD")
else:
    print("Neo4j credentials configured")

Neo4j credentials configured


In [5]:
databricks_token = os.getenv("DATABRICKS_TOKEN")
databricks_host = os.getenv("DATABRICKS_HOST")
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow_registry_uri = os.getenv("MLFLOW_REGISTRY_URI")
mlflow_experiment_id = os.getenv("MLFLOW_EXPERIMENT_ID")

if not all([databricks_token, databricks_host, mlflow_tracking_uri, mlflow_registry_uri,mlflow_experiment_id]):
    print("WARNING: Databricks / MLFLOW  credentials not found")
    print("Please set DATABRICKS_TOKEN, DATABRICKS_HOST, MLFLOW_TRACKING_URI, MLFLOW_REGISTRY_URI and MLFLOW_EXPERIMENT_ID")
else:
    print("Databricks / MLFLOW credentials configured")

Databricks / MLFLOW credentials configured


In [6]:
# Validate required environment variables (aligned with Key Vault secret map)
from pprint import pprint

required_env_vars = [
    "AZURE_CLIENT_ID",
    "AZURE_CLIENT_SECRET",
    "AZURE_TENANT_ID",
    "SUBSCRIPTION_ID",
    "RESOURCE_GROUP",
    "AZUREML_WORKSPACE_NAME",
    "KEYVAULT_NAME",
    "OPENAI_API_KEY",
    "AZURE_API_KEY",
    "AZURE_ENDPOINT",
    "AZURE_DEPLOYMENT",
    "AZURE_API_VERSION",
    "NEO4J_URI",
    "NEO4J_USERNAME",
    "NEO4J_PASSWORD",
    "DATABRICKS_TOKEN",
    "DATABRICKS_HOST",
    "MLFLOW_TRACKING_URI",
    "MLFLOW_REGISTRY_URI",
    "MLFLOW_EXPERIMENT_ID",
    "APPLICATIONINSIGHTS_CONNECTION_STRING",
    "PA_ENV_VERSION",
]

missing_env = [var for var in required_env_vars if not os.getenv(var)]
if missing_env:
    print("WARNING: The following environment variables are missing. Confirm notebooks/.env or Key Vault entries:")
    pprint(missing_env)
else:
    print("All required environment variables are present.")

All required environment variables are present.


In [7]:
# Environment variables propagated to every pipeline step
current_dir = Path.cwd()
project_root_for_env = current_dir.parent if current_dir.name == "notebooks" else current_dir
pythonpath_entries = [
    project_root_for_env.as_posix(),
    (project_root_for_env / "PA").as_posix(),
]
path_separator = ":" if os.name != "nt" else ";"
pythonpath_value = path_separator.join([entry for entry in pythonpath_entries if entry])
keyvault_uri = None
if os.getenv("KEYVAULT_NAME"):
    keyvault_uri = f"https://{os.getenv('KEYVAULT_NAME')}.vault.azure.net/"
pa_env_version = os.getenv("PA_ENV_VERSION", str(DEFAULT_PA_ENV_VERSION))
otel_resource_detectors = os.getenv("OTEL_EXPERIMENTAL_RESOURCE_DETECTORS", "otel")
otel_service_name = os.getenv("OTEL_SERVICE_NAME", "pa-azureml-step")
env_vars_template = {
    "AZURE_CLIENT_ID": os.getenv("AZURE_CLIENT_ID"),
    "AZURE_CLIENT_SECRET": os.getenv("AZURE_CLIENT_SECRET"),
    "AZURE_TENANT_ID": os.getenv("AZURE_TENANT_ID"),
    "SUBSCRIPTION_ID": os.getenv("SUBSCRIPTION_ID"),
    "RESOURCE_GROUP": os.getenv("RESOURCE_GROUP"),
    "AZUREML_WORKSPACE_NAME": os.getenv("AZUREML_WORKSPACE_NAME"),
    "STORAGE_ACCOUNT_NAME": "strategicaistuksdev02",
    "KEYVAULT_NAME": os.getenv("KEYVAULT_NAME"),
    "KEYVAULT_URI": keyvault_uri,
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY"),
    "AZURE_API_KEY": os.getenv("AZURE_API_KEY"),
    "AZURE_ENDPOINT": os.getenv("AZURE_ENDPOINT"),
    "AZURE_DEPLOYMENT": os.getenv("AZURE_DEPLOYMENT"),
    "AZURE_API_VERSION": os.getenv("AZURE_API_VERSION"),
    "AZURE_OPENAI_API_KEY": os.getenv("AZURE_OPENAI_API_KEY"),
    "AZURE_OPENAI_ENDPOINT": os.getenv("AZURE_OPENAI_ENDPOINT"),
    "AZURE_DEPLOYMENT_BATCH": os.getenv("AZURE_DEPLOYMENT_BATCH"),
    "AZURE_API_VERSION_BATCH": os.getenv("AZURE_API_VERSION_BATCH"),
    "NEO4J_URI": os.getenv("NEO4J_URI"),
    "NEO4J_USERNAME": os.getenv("NEO4J_USERNAME"),
    "NEO4J_PASSWORD": os.getenv("NEO4J_PASSWORD"),
    "NEO4J_URI_DEV": os.getenv("NEO4J_URI_DEV"),
    "NEO4J_PASSWORD_DEV": os.getenv("NEO4J_PASSWORD_DEV"),
    "NEO4J_URI_TEST": os.getenv("NEO4J_URI_TEST"),
    "NEO4J_PASSWORD_TEST": os.getenv("NEO4J_PASSWORD_TEST"),
    "NEO4J_URI_PROD": os.getenv("NEO4J_URI_PROD"),
    "NEO4J_PASSWORD_PROD": os.getenv("NEO4J_PASSWORD_PROD"),
    "DATABRICKS_TOKEN": os.getenv("DATABRICKS_TOKEN"),
    "DATABRICKS_HOST": os.getenv("DATABRICKS_HOST"),
    "DATABRICKS_HOSTS": os.getenv("DATABRICKS_HOSTS"),
    "MLFLOW_TRACKING_URI": os.getenv("MLFLOW_TRACKING_URI"),
    "MLFLOW_REGISTRY_URI": os.getenv("MLFLOW_REGISTRY_URI"),
    "MLFLOW_EXPERIMENT_ID": os.getenv("MLFLOW_EXPERIMENT_ID"),
    "APPLICATIONINSIGHTS_CONNECTION_STRING": os.getenv("APPLICATIONINSIGHTS_CONNECTION_STRING"),
    "AZURE_APPINSIGHTS_CONNECTION_STRING": os.getenv("AZURE_APPINSIGHTS_CONNECTION_STRING"),
    "APPINSIGHTS_CONNECTION_STRING": os.getenv("APPINSIGHTS_CONNECTION_STRING"),
    "APPLICATIONINSIGHTS_AUTHENTICATION_MODE": os.getenv("APPLICATIONINSIGHTS_AUTHENTICATION_MODE"),
    "OTEL_EXPERIMENTAL_RESOURCE_DETECTORS": otel_resource_detectors,
    "OTEL_SERVICE_NAME": otel_service_name,
    "PA_ENV_VERSION": pa_env_version,
    "PYTHONPATH": pythonpath_value,
}

# Remove keys without values (Azure ML expects string values only)
environment_variables = {key: value for key, value in env_vars_template.items() if value not in (None, "")}

print(f"Propagating {len(environment_variables)} environment variables to every step")

Propagating 40 environment variables to every step


In [8]:
# Create ML Client
credential = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret
)

ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name
)

print(f"Connected to workspace: {workspace_name}")

Connected to workspace: strategicai-mlw-uks-dev-01


In [9]:
# Verify project structure
current_dir = Path.cwd()
if current_dir.name == "notebooks":
    project_root = current_dir.parent
    print(f"Running from notebooks folder")
else:
    project_root = current_dir
    print(f"Running from project root")

print(f"Project root: {project_root}")

# Check required directories
pa_dir = project_root / "PA"
if not pa_dir.exists():
    print(f"ERROR: PA directory not found at {pa_dir}")
else:
    print(f"✓ PA directory found")

pipeline_dir = project_root / "azureml_pipeline"
if not pipeline_dir.exists():
    print(f"Creating azureml_pipeline directory")
    pipeline_dir.mkdir(exist_ok=True)
else:
    print(f"✓ azureml_pipeline directory found")

# Check for all step scripts
step_scripts = {
    "Step 1": pipeline_dir / "azureml_step1_data_prep.py",
    "Step 2": pipeline_dir / "azureml_step2_neo4j_prep.py",
    "Step 3": pipeline_dir / "azureml_step3_session_embedding.py",
    "Step 4": pipeline_dir / "azureml_step4_recommendations.py"
}

for step_name, script_path in step_scripts.items():
    if script_path.exists():
        print(f"✓ {step_name} script found")
    else:
        print(f"WARNING: {step_name} script not found at {script_path}")

# Check config files
config_vet = pa_dir / "config" / "config_vet_lva.yaml"
config_ecomm = pa_dir / "config" / "config_ecomm.yaml"

if config_ecomm.exists():
    print(f"✓ config_ecomm.yaml found")
if config_vet.exists():
    print(f"✓ config_vet_lva.yaml found")

Running from notebooks folder
Project root: /mnt/batch/tasks/shared/LS_root/mounts/clusters/vm-juan/code/Notebooks/repos/azure_ml_pipelines_pa
✓ PA directory found
✓ azureml_pipeline directory found
✓ Step 1 script found
✓ Step 2 script found
✓ Step 3 script found
✓ Step 4 script found
✓ config_ecomm.yaml found
✓ config_vet_lva.yaml found


In [ ]:
# Environment setup (restored auto-version logic)
dependencies_dir = Path("./env")
if not dependencies_dir.exists():
    notebooks_env = project_root / "notebooks" / "env"
    repo_env = project_root / "env"
    if notebooks_env.exists():
        dependencies_dir = notebooks_env
    elif repo_env.exists():
        dependencies_dir = repo_env
custom_env_name = "pa-env"
requested_version_raw = os.getenv("PA_ENV_VERSION", str(DEFAULT_PA_ENV_VERSION))
try:
    requested_env_version = int(requested_version_raw)
except ValueError:
    print(
        f"WARNING: Invalid PA_ENV_VERSION '{requested_version_raw}'. Will auto-increment from existing versions.")
    requested_env_version = None

existing_envs = {}
existing_versions = []
try:
    for env in ml_client.environments.list(name=custom_env_name):
        version_str = str(env.version)
        if version_str.isdigit():
            version_int = int(version_str)
            existing_versions.append(version_int)
            existing_envs[version_int] = env
except Exception as exc:
    print(f"Warning: unable to enumerate existing environments ({exc})")

latest_version = max(existing_versions) if existing_versions else None

conda_file = dependencies_dir / "conda.yaml"
if not conda_file.exists():
    print(f"WARNING: conda.yaml not found at {conda_file}")
    print("Using default conda configuration")

if requested_env_version is not None and requested_env_version in existing_envs:
    job_env = existing_envs[requested_env_version]
    print(f"Using existing environment: {job_env.name}:{job_env.version}")
elif latest_version is not None and requested_env_version == latest_version:
    job_env = existing_envs[latest_version]
    print(
        f"Notebook version matches latest workspace version ({latest_version}); reusing environment {job_env.name}:{job_env.version}")
else:
    version_to_create = (
        requested_env_version
        if requested_env_version is not None
        else (latest_version + 1 if latest_version is not None else 1)
    )

    if latest_version is not None and version_to_create <= latest_version:
        version_to_create = latest_version + 1
        print(f"Requested version already exists or is older. Incrementing to {version_to_create}.")

    print(f"Creating environment version {version_to_create} for {custom_env_name}")
    environment_kwargs = {
        "name": custom_env_name,
        "version": str(version_to_create),
        "description": "Environment for Personal Agendas pipeline with Neo4j and embeddings",
        "conda_file": str(conda_file) if conda_file.exists() else None,
        "image": "mcr.microsoft.com/azureml/openmpi5.0-ubuntu24.04:20250601.v1",
    }

    job_env = Environment(**environment_kwargs)
    job_env = ml_client.environments.create_or_update(job_env)
    print(f"Created environment: {job_env.name}:{job_env.version}")

In [ ]:
# Define INCREMENTAL versions of the components (optional)
session_embedding_component_incremental = command(
    name="session_embedding_incremental",
    display_name="Step 3: Session Embedding (Incremental)",
    description="Generate embeddings only for new sessions without embeddings",
    inputs={
        "config_type": Input(type="string", default="ecomm"),
        "neo4j_ready": Input(type="uri_folder", optional=True),
        "input_registration": Input(type="uri_folder"),
        "input_scan": Input(type="uri_folder"),
        "input_session": Input(type="uri_folder"),
    },
    outputs={
        "metadata_output": Output(type="uri_folder")
    },
    code=str(project_root),
    command="""python azureml_pipeline/azureml_step3_session_embedding.py \
        --config PA/config/config_${{inputs.config_type}}.yaml \
        --input_registration ${{inputs.input_registration}} \
        --input_scan ${{inputs.input_scan}} \
        --input_session ${{inputs.input_session}} \
        --incremental true \
        --output_metadata ${{outputs.metadata_output}}
    """,
    environment=f"{job_env.name}:{job_env.version}",
    environment_variables=environment_variables,
    compute="cpu-cluster",
    is_deterministic=False
)


Requested PA_ENV_VERSION 10 is lower than supported minimum 12. Forcing 12.
Pinned 59 pip dependencies into /mnt/batch/tasks/shared/LS_root/mounts/clusters/vm-juan/code/Notebooks/repos/azure_ml_pipelines_pa/azureml_pipeline/pa_env_v12_conda.yaml
Created environment: pa-env:12
Pipeline steps will use environment version 12


In [11]:
# Step 1: Data Preparation Component
data_preparation_component = command(
    name="data_preparation",
    display_name="Step 1: Data Preparation",
    description="Process registration, scan, and session data",
    inputs={
        "input_uri": Input(type="uri_folder"),
        "config_type": Input(type="string", default="ecomm"),
        "incremental": Input(type="string", default="false"),
    },
    outputs={
        "registration_output": Output(type="uri_folder"),
        "scan_output": Output(type="uri_folder"),
        "session_output": Output(type="uri_folder"),
        "metadata_output": Output(type="uri_folder")
    },
    code=str(project_root),
    command="""python azureml_pipeline/azureml_step1_data_prep.py \\
        --config PA/config/config_${{inputs.config_type}}.yaml \\
        --input_uri ${{inputs.input_uri}} \\
        --incremental ${{inputs.incremental}} \\
        --output_registration ${{outputs.registration_output}} \\
        --output_scan ${{outputs.scan_output}} \\
        --output_session ${{outputs.session_output}} \\
        --output_metadata ${{outputs.metadata_output}}
    """,
    environment=f"{job_env.name}:{job_env.version}",
    environment_variables=environment_variables,
    compute="cpu-cluster",
    is_deterministic=False
)

In [12]:
# Step 2: Neo4j Preparation Component

neo4j_preparation_component = command(
    name="neo4j_preparation",
    display_name="Step 2: Neo4J Preparation",
    description="Upload data to Neo4j database",
    inputs={
        "config_type": Input(type="string", default="ecomm"),
        "input_registration": Input(type="uri_folder"),
        "input_scan": Input(type="uri_folder"),
        "input_session": Input(type="uri_folder"),
        "incremental": Input(type="string", default="false"),
    },
    outputs={
        "metadata_output": Output(type="uri_folder")
    },
    code=str(project_root),
    command="""python azureml_pipeline/azureml_step2_neo4j_prep.py \\
        --config PA/config/config_${{inputs.config_type}}.yaml \\
        --input_registration ${{inputs.input_registration}} \\
        --input_scan ${{inputs.input_scan}} \\
        --input_session ${{inputs.input_session}} \\
        --incremental ${{inputs.incremental}} \\
        --output_metadata ${{outputs.metadata_output}}
    """,
    environment=f"{job_env.name}:{job_env.version}",
    environment_variables=environment_variables,
    compute="cpu-cluster",
    is_deterministic=False
)

print("✓ Step 2 component defined")

✓ Step 2 component defined


In [ ]:
# Step 3: Session Embedding Component
session_embedding_component = command(
    name="session_embedding",
    display_name="Step 3: Session Embedding",
    description="Generate and store text embeddings for session nodes in Neo4j",
    inputs={
        "config_type": Input(type="string", default="ecomm"),
        "neo4j_ready": Input(type="uri_folder", optional=True),
        "input_registration": Input(type="uri_folder"),
        "input_scan": Input(type="uri_folder"),
        "input_session": Input(type="uri_folder"),
        "incremental": Input(type="string", default="false"),
    },
    outputs={
        "metadata_output": Output(type="uri_folder")
    },
    code=str(project_root),
    command="""python azureml_pipeline/azureml_step3_session_embedding.py \
        --config PA/config/config_${{inputs.config_type}}.yaml \
        --input_registration ${{inputs.input_registration}} \
        --input_scan ${{inputs.input_scan}} \
        --input_session ${{inputs.input_session}} \
        --incremental ${{inputs.incremental}} \
        --output_metadata ${{outputs.metadata_output}}
    """,
    environment=f"{job_env.name}:{job_env.version}",
    environment_variables=environment_variables,
    compute="cpu-cluster",
    is_deterministic=False
)

print("✓ Step 3 component defined")

✓ Step 3 component defined


In [ ]:
# Step 4: Recommendations Component
recommendations_component = command(
    name="recommendations",
    display_name="Step 4: Recommendations",
    description="Generate session recommendations for visitors",
    inputs={
        "config_type": Input(type="string", default="ecomm"),
        "embeddings_ready": Input(type="uri_folder", optional=True),
        "input_registration": Input(type="uri_folder"),
        "input_scan": Input(type="uri_folder"),
        "input_session": Input(type="uri_folder"),
        "incremental": Input(type="string", default="false"),
    },
    outputs={
        "metadata_output": Output(type="uri_folder")
    },
    code=str(project_root),
    command="""python azureml_pipeline/azureml_step4_recommendations.py \
        --config PA/config/config_${{inputs.config_type}}.yaml \
        --input_registration ${{inputs.input_registration}} \
        --input_scan ${{inputs.input_scan}} \
        --input_session ${{inputs.input_session}} \
        --incremental ${{inputs.incremental}} \
        --output_metadata ${{outputs.metadata_output}}
    """,
    environment=f"{job_env.name}:{job_env.version}",
    environment_variables=environment_variables,
    compute="cpu-cluster",
    is_deterministic=False
)

print("✓ Step 4 component defined")

✓ Step 4 component defined


In [ ]:
# Define the complete pipeline
@pipeline(
    compute="cpu-cluster",
    description="Complete Personal Agendas pipeline with 4 steps",
)
def personal_agendas_complete_pipeline(
    pipeline_input_data: Input,
    pipeline_config_type: str = "ecomm",
    pipeline_incremental: str = "false"
):
    """Complete Personal Agendas pipeline."""
    
    # Step 1: Data Preparation
    step1 = data_preparation_component(
        input_uri=pipeline_input_data,
        config_type=pipeline_config_type,
        incremental=pipeline_incremental
    )
    step1.name = "step1_data_preparation"
    
    # Step 2: Neo4J Preparation - uses outputs from Step 1
    step2 = neo4j_preparation_component(
        config_type=pipeline_config_type,
        input_registration=step1.outputs.registration_output,
        input_scan=step1.outputs.scan_output,
        input_session=step1.outputs.session_output,
        incremental=pipeline_incremental
    )
    step2.name = "step2_neo4j_preparation"
    
    # Step 3: Session Embedding - runs after Neo4j data is loaded
    # It stages Step 1 artifacts for any processors that expect files on disk
    step3 = session_embedding_component(
        config_type=pipeline_config_type,
        neo4j_ready=step2.outputs.metadata_output,
        input_registration=step1.outputs.registration_output,
        input_scan=step1.outputs.scan_output,
        input_session=step1.outputs.session_output,
        incremental=pipeline_incremental
    )
    step3.name = "step3_session_embedding"
    
    # Step 4: Recommendations
    step4 = recommendations_component(
        config_type=pipeline_config_type,
        embeddings_ready=step3.outputs.metadata_output,
        input_registration=step1.outputs.registration_output,
        input_scan=step1.outputs.scan_output,
        input_session=step1.outputs.session_output,
        incremental=pipeline_incremental
    )
    step4.name = "step4_recommendations"
    
    # Return all outputs
    return {
        "registration_data": step1.outputs.registration_output,
        "scan_data": step1.outputs.scan_output,
        "session_data": step1.outputs.session_output,
        "step1_metadata": step1.outputs.metadata_output,
        "neo4j_metadata": step2.outputs.metadata_output,
        "embedding_metadata": step3.outputs.metadata_output,
        "recommendations_metadata": step4.outputs.metadata_output
    }

print("✓ Complete pipeline defined")

✓ Complete pipeline defined


In [16]:
# Configure input data URI
input_data_uri = f"azureml://subscriptions/{subscription_id}/resourcegroups/{resource_group}/workspaces/{workspace_name}/datastores/landing_pa/paths/landing/azureml/"

print(f"Input data URI: {input_data_uri}")

Input data URI: azureml://subscriptions/b8d6d487-0bd2-4773-b318-12ab763ed178/resourcegroups/strategicai-rg-uks-dev-01/workspaces/strategicai-mlw-uks-dev-01/datastores/landing_pa/paths/landing/azureml/


In [17]:
# Create pipeline instance
pipeline_incremental_value = "false"
pipeline_job = personal_agendas_complete_pipeline(
    pipeline_input_data=Input(
        type=AssetTypes.URI_FOLDER,
        path=input_data_uri
    ),
    pipeline_config_type="vet_lva",  # or "vet"
    pipeline_incremental=pipeline_incremental_value
)

# Configure pipeline metadata
pipeline_job.display_name = "Personal Agendas Complete Pipeline - LVA"
pipeline_job.tags = {
    "project": "personal_agendas",
    "event_type": "lva",
    "environment": "prod",
    "includes_neo4j": "true",
    "includes_embeddings": "true",
    "includes_recommendations": "true",
    "incremental": pipeline_incremental_value,
    "step_count": "4",
    "complete_pipeline": "true"
}
pipeline_job.experiment_name = "personal_agendas_complete_experiment"

print("✓ Pipeline instance created")

✓ Pipeline instance created


In [18]:
# Submit the pipeline
print("\nSubmitting pipeline...")
print("="*60)

try:
    # Submit the pipeline job
    pipeline_run = ml_client.jobs.create_or_update(pipeline_job)
    
    print(f"✓ Pipeline submitted successfully!")
    print(f"\nPipeline Details:")
    print(f"  Name: {pipeline_run.name}")
    print(f"  Display Name: {pipeline_run.display_name}")
    print(f"  Status: {pipeline_run.status}")
    print(f"  Experiment: {pipeline_run.experiment_name}")
    print(f"\n🔗 View pipeline in Azure ML Studio:")
    print(f"  {pipeline_run.studio_url}")
    
except Exception as e:
    print(f"❌ Error submitting pipeline: {str(e)}")
    raise

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.



Submitting pipeline...


Uploading azure_ml_pipelines_pa (2.61 MBs): 100%|██████████| 2607062/2607062 [00:01<00:00, 1750479.38it/s]


Uploading azure_ml_pipelines_pa (2.61 MBs): 100%|██████████| 2605010/2605010 [00:01<00:00, 1838004.89it/s]


pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> an

✓ Pipeline submitted successfully!

Pipeline Details:
  Name: quirky_peach_6xd1z4p9w8
  Display Name: Personal Agendas Complete Pipeline - LVA
  Status: NotStarted
  Experiment: personal_agendas_complete_experiment

🔗 View pipeline in Azure ML Studio:
  https://ml.azure.com/runs/quirky_peach_6xd1z4p9w8?wsid=/subscriptions/b8d6d487-0bd2-4773-b318-12ab763ed178/resourcegroups/strategicai-rg-uks-dev-01/workspaces/strategicai-mlw-uks-dev-01&tid=3540e7dc-31b3-4057-9e31-43e9fe938179


In [ ]:
# Optional: Monitor pipeline progress
import time

print("\nMonitoring pipeline progress...")
print("(Press Ctrl+C to stop monitoring)\n")

try:
    while True:
        # Get the latest status
        job = ml_client.jobs.get(pipeline_run.name)
        
        print(f"[{datetime.now().strftime('%H:%M:%S')}] Status: {job.status}", end="")
        
        if job.status in ["Completed", "Failed", "Canceled"]:
            print(f"\n\nPipeline {job.status}!")
            if job.status == "Failed":
                print("Check the logs in Azure ML Studio for error details.")
            break
        
        print(" (waiting...)", end="\r")
        time.sleep(30)  # Check every 30 seconds
        
except KeyboardInterrupt:
    print("\n\nMonitoring stopped. Pipeline continues running in Azure ML.")
    print(f"Check status at: {pipeline_run.studio_url}")


Monitoring pipeline progress...
(Press Ctrl+C to stop monitoring)

[17:24:54] Status: NotStarted (waiting...)

## Pipeline Summary

This notebook submits a complete 4-step Personal Agendas pipeline:

1. **Data Preparation**: Processes registration, scan, and session data
2. **Neo4j Preparation**: Loads data into Neo4j and creates relationships
3. **Session Embedding**: Generates embeddings for sessions
4. **Recommendations**: Generates personalized session recommendations

### Key Features:
- Fully generic and configurable (works with both `ecomm` and `vet` configurations)
- Proper data flow between steps
- Neo4j integration with secure credential management
- Incremental processing support (can be enabled per step)
- Complete error handling and logging

### Next Steps:
1. Monitor the pipeline execution in Azure ML Studio
2. Check the output metadata for each step
3. Verify recommendations are generated in Neo4j
4. Download the recommendations output file for analysis